# Sample a very large CSV file using pandas

The following shows two ways to sample a large CSV file to get the desired number of rows as a subset.
This is useful for getting a sample for understanding the data, and doing fast manipulations prior to applying logic across an entire file.

The two approaches are as follows:
1) Get the first x rows in a file - very, very fast, but not random
2) Randomly select x rows from the file

## Set up

In [23]:
# Import the necessary libraries
import pandas as pd
import numpy as np
import datetime

# Select the sample size, in rows, that we want
sample_size = 5000

# define input and output file names
huge_file_name = 'hugefile.csv'
first_x_rows_file_name = 'firstxrows.csv'
random_sample_file_name = 'randomsample.csv'

## Retrieve the first x rows in a file

This is very, very fast and simple. But the data will likely not represent the overall dataset appropriately. That may not be a problem, but it depends strongly on what you'll be doing with the sample.

In [24]:
# read the CSV data, passing in nrows as a parameter to get just that number of rows (from the beginning)
df = pd.read_csv(huge_file_name, sep=",", low_memory=False, nrows=sample_size)


In [25]:
# Save the partial sample as CSV
df.to_csv(first_x_rows_file_name, sep=",", index=False)

## Randomly select x rows in a file

This is very, very slow with pandas and not as simple. But the data will be more likely to represent the actual dataset. 

The problem is that pandas.read_csv only has a skiprows parameter, not a takerows parameter.  Which means that you have to create a massive array of random numbers for lines to skip (that don't have duplicates) to get a small number of rows.  For instance, in my current dataset I have approximately 20 million rows.  Of which I want to pull out only 5000.  So I have to create an array of 1,995,000 random numbers. This will define the rows NOT taken.

Thanks to http://stackoverflow.com/users/2643104/queise for the idea on StackOverflow: http://stackoverflow.com/questions/22258491/read-a-small-random-sample-from-a-big-csv-file-into-a-python-data-frame

There are other, faster and potentially cleaner, ways to get a small subset from a document.

1) Convert to HDF5 and us pandas.read_hdf method, which allows for sampling directly: http://stackoverflow.com/questions/21039772/pytables-read-random-subset

2) Read through the file and randomly select rows to add to another file in pure Python. http://stackoverflow.com/questions/10819911/read-random-lines-from-huge-csv-file-in-python

However, I stuck with the pandas.read_csv approach because it's only run rarely, so I don't care if it takes a long time. This isn't part of a pipeline for me, just a one-off every now and again to grab data.

In [26]:
# First determine the number of rows in the file
#   Iterate through all rows and sum
#   Also, time how long it takes, just for FYI
print(datetime.datetime.now().time()) # start time
with open(huge_file_name) as f:
    total_rows = sum(1 for _ in f)
print(datetime.datetime.now().time()) # end time
print(total_rows)

18:03:33.943099
18:04:28.113050
20841012


In [34]:
# create a list of size = num_rows - sample_size of random numbers
#   I'm always including the first row, since I'm making the assumption that contains header information
lines2skip = np.random.choice(np.arange(1,total_rows+1), (total_rows-sample_size), replace=False)
print("Number of rows to skip: {0}".format(len(lines2skip)))

print(datetime.datetime.now().time()) # start time
df = pd.read_csv(huge_file_name, sep=",", low_memory=False, skiprows=lines2skip)
print(datetime.datetime.now().time()) # end time
print("Number of rows in DataFrame: {0}".format(len(df.index)))

Number of rows to skip: 20836012
18:11:06.165123
18:11:41.467551
Number of rows in DataFrame: 


In [35]:
# Save the partial sample as CSV
df.to_csv(random_sample_file_name, sep=",", index=False)